In [469]:
import time
start = time.time()
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import glob
print(time.time()-start,'s')

0.0004646778106689453 s


### Algorithm 1

In [477]:
# image = cv2.imread('photo10.jpg')
# gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [478]:
# height, width = gray.shape
# print(height)
# print(width)
# out = cv2.rectangle(gray,(int(width/10),int(17*height/23)),(int(10*width/11),int(9*height/10)),(0,255,0),2)
# # out = cv2.rectangle(gray,(int(width/10),int(18*height/25)),(int(15*width/16),int(13*height/14)),(0,255,0),2)
# cv2.imwrite('Out.jpg',out)

1280
900


True

### Algorithm 2

In [472]:
main = cv2.imread('photo10.jpg')
crop = cv2.imread('column.jpg')
gray_main = cv2.cvtColor(main,cv2.COLOR_BGR2GRAY)
gray_crop = cv2.cvtColor(crop,cv2.COLOR_BGR2GRAY)
height_main, width_main = gray_main.shape
height_crop, width_crop = gray_crop.shape
height = height_main
width = width_main
print(height_main, width_main)
print(height_crop, width_crop)
test = gray_main

1280 900
316 1280


In [473]:
#Trying goodFeatures to Track
# corners = cv2.goodFeaturesToTrack(gray_main,20,0.001,50)
# corners = np.int0(corners)

# for i in corners:
#     x,y = i.ravel()
#     cv2.circle(test,(x,y),10,0,-1)

# cv2.imwrite('Out.jpg',test)

#### Approach 1
Better accuracy as of now.

In [474]:
#Trying feature matching using ORB
orb = cv2.ORB_create()

mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)
cv2.rectangle(mask, (int(width/10),int(18*height/25)),(int(15*width/16),int(13*height/14)), (255), thickness = -1)
# find the keypoints and descriptors with SIFT
kp1, des1 = orb.detectAndCompute(gray_main,mask)
kp2, des2 = orb.detectAndCompute(gray_crop,None)


In [476]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

matches = bf.match(des1,des2)

matches = sorted(matches, key = lambda x:x.distance)

img = cv2.drawMatches(gray_main,kp1,gray_crop,kp2,matches,None, flags=2)

cv2.imwrite('Out.jpg',img)

True

#### Approach 2 

In [459]:
# main = cv2.imread('photo10.jpg')
# crop = cv2.imread('column.jpg')
# gray_main = cv2.cvtColor(main,cv2.COLOR_BGR2GRAY)
# gray_crop = cv2.cvtColor(crop,cv2.COLOR_BGR2GRAY)
# height_main, width_main = gray_main.shape
# height_crop, width_crop = gray_crop.shape
# print(height_main, width_main)
# print(height_crop, width_crop)

In [460]:
# #Trying feature matching using SIFT
# # Initiate SIFT detector
# sift = cv2.xfeatures2d.SIFT_create()

# mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)
# cv2.rectangle(mask, (int(width/10),int(18*height/25)),(int(15*width/16),int(13*height/14)), (255), thickness = -1)

# # find the keypoints and descriptors with SIFT
# kp1, des1 = sift.detectAndCompute(gray_main,None)
# kp2, des2 = sift.detectAndCompute(gray_crop,None)

# FLANN_INDEX_KDTREE = 0
# index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
# search_params = dict(checks = 50)

# flann = cv2.FlannBasedMatcher(index_params, search_params)

# matches = flann.knnMatch(des1,des2,k=2,mask=mask)

# # store all the good matches as per Lowe's ratio test.
# good = []
# for m,n in matches:
#     if m.distance < 0.8*n.distance:
#         good.append(m)

In [461]:
# src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
# dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

# M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
# matchesMask = mask.ravel().tolist()

# h,w = gray_main.shape
# pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
# dst = cv2.perspectiveTransform(pts,M)


# img2 = cv2.polylines(gray_crop,[np.int32(dst)],True,255,3, cv2.LINE_AA)

In [462]:
# draw_params = dict(matchColor = (0,255,0), # draw matches in green color
#                    singlePointColor = None,
#                    matchesMask = matchesMask, # draw only inliers
#                    flags = 2)

# img3 = cv2.drawMatches(gray_main,kp1,gray_crop,kp2,good,None,**draw_params)
# cv2.imwrite('Out.jpg',img3)

In [463]:
#points = src_pts.reshape(-1,2)
# src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ])
src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ])
points = src_pts

In [464]:
points.shape

(224, 2)

In [2]:
def get_outer_points(pts):

    rect = np.zeros((4, 2), dtype = "float32")
 
    min_x = min(pts[:,0])
    min_y = min(pts[:,1])
    max_x = max(pts[:,0])
    max_y = max(pts[:,1])
    #s = pts.sum(axis = 1)
    #print(s)
    rect[0][0] = min_x
    rect[0][1] = min_y
    rect[2][0] = max_x
    rect[2][1] = max_y
 

    #diff = np.diff(pts, axis = 1)
    #print(diff)
    rect[1][0] = max_x
    rect[1][1] = min_y
    rect[3][0] = min_x
    rect[3][1] = max_y
 
    return rect

In [3]:
def four_point_transform(image, pts):

    rect = get_outer_points(pts)
    (tl, tr, br, bl) = rect
    print(rect)

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
              [0, 0],
              [maxWidth - 1, 0],
              [maxWidth - 1, maxHeight - 1],
              [0, maxHeight - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    cv2.imwrite('Out1.jpg',warped)
 
    return warped

In [1]:
crop = 10
dst = four_point_transform(gray_main, points)
dst_h, dst_w = dst.shape[:2]
img = dst[crop:dst_h+crop, crop:dst_w+crop]
dst_h, dst_w = img.shape[:2]
cv2.imwrite('Out1.jpg',img)

NameError: name 'four_point_transform' is not defined

In [468]:
time.time()-start

2.5622060298919678